In [ ]:
  !pip install netCDF4
!pip3 install numpy pylibtiff
!pip install georasters

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import netCDF4
import numpy as np
import string

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid

import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plot
%matplotlib inline

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

ws = gc.open('map data').sheet1
# link to sheet: https://docs.google.com/spreadsheets/d/17bffZenh_TK5gDn0CoppMLiBpgf6r3f7MCrW1iB1j7g/edit?usp=sharing

from libtiff import TIFF
import georasters as gr
import xarray as xr

In [ ]:
def find_names(start_file_name):
  arr=[]
  currName=start_file_name
  currYr=1992
  maxYr=2016
  ran=maxYr-currYr
  for i in range(ran+1):
    arr.append(currName)
    currName = currName.replace(str(currYr), str(currYr+1))
    currYr+=1
  return np.array(arr)

In [ ]:
def calc_bounds(files, type):
  global_var_min=10e9
  global_var_max=10e-9
  for i in files:
    nc = netCDF4.Dataset(i)
    time = nc.variables['time'][:]
    for time_index in range(len(time)):
      var = nc.variables[type][:]
      global_var_min=min(global_var_min, var.min())
      global_var_max=max(global_var_max, var.max())
  return global_var_min,global_var_max

In [ ]:
def calc_bounds_crop(files):
  global_var_min=10e9
  global_var_max=10e-9
  for i in files:
    nc = netCDF4.Dataset(i)
    yield_at_time = nc.variables['var'][:]
    global_var_min=min(global_var_min, yield_at_time.min())
    global_var_max=max(global_var_max, yield_at_time.max())
  return global_var_min,global_var_max

In [ ]:
agriculture = []
files = find_names("/content/drive/MyDrive/crop_yield/wheat/yield_1992.nc4")
vminfound,vmaxfound=calc_bounds_crop(files)
for i in range(1992, 2017):
  nc = netCDF4.Dataset("/content/drive/MyDrive/crop_yield/wheat/yield_"+str(i)+".nc4")
  lon1 = nc.variables['lon'][:]
  lat1 = nc.variables['lat'][:]
  yield_at_time1 = nc.variables['var'][:]
  plt.figure(figsize=(10, 10))
  plt.imshow(yield_at_time1, origin='lower', extent=[lat1.min(), lat1.max(), lon1.min(), lon1.max(),], aspect='auto', cmap='coolwarm', vmin=vminfound, vmax=vmaxfound)
  plt.title("Yield in year "+str(i))
  plt.xlabel('Latitude')
  plt.ylabel('Longitude')
  plt.colorbar(label='Yield')
  agriculture.append(plt.gcf())
  plt.close()

In [ ]:
precipitation = []
files = find_names("/content/drive/MyDrive/temp_and_precip/CRU_total_precipitation_mon_0.5x0.5_global_1992_v4.03.nc")
vminfound,vmaxfound=calc_bounds(files,'pr')
for i in range(1992, 2017):
  nc = netCDF4.Dataset("/content/drive/MyDrive/temp_and_precip/CRU_total_precipitation_mon_0.5x0.5_global_"+str(i)+"_v4.03.nc")
  time = nc.variables['time'][:]
  lon = nc.variables['lon'][:]
  lat = nc.variables['lat'][:]
  varPrecip = nc.variables['pr'][:]
  pr_at_time = varPrecip[0, :, :]
  plt.figure(figsize=(10, 10))
  plt.imshow(pr_at_time, origin='lower', extent=[lat.min(), lat.max(), lon.min(), lon.max()], aspect='auto', cmap='coolwarm', vmin=vminfound, vmax=vmaxfound)
  plt.title("Precipitation in year "+str(i))
  plt.xlabel('Latitude')
  plt.ylabel('Longitude')
  plt.colorbar(label='Precipitation')
  precipitation.append(plt.gcf())
  plt.close()

In [ ]:
temperature = []
files = find_names("/content/drive/MyDrive/temp_and_precip/CRU_mean_temperature_mon_0.5x0.5_global_1992_v4.03.nc")
vminfound,vmaxfound=calc_bounds(files,'tas')
for i in range(1992, 2017):
  nc = netCDF4.Dataset("/content/drive/MyDrive/temp_and_precip/CRU_mean_temperature_mon_0.5x0.5_global_"+str(i)+"_v4.03.nc")
  time = nc.variables['time'][:]
  lon = nc.variables['lon'][:]
  lat = nc.variables['lat'][:]
  varTemp = nc.variables['tas'][:]
  tas_at_time = varTemp[0, :, :]
  plt.figure(figsize=(10, 10))
  plt.imshow(tas_at_time, origin='lower', extent=[lat.min(), lat.max(), lon.min(), lon.max()], aspect='auto', cmap='coolwarm', vmin=vminfound, vmax=vmaxfound)
  plt.title("Temperature in year "+str(i))
  plt.xlabel('Latitude')
  plt.ylabel('Longitude')
  plt.colorbar(label='Temperature')
  temperature.append(plt.gcf())
  plt.close()

In [ ]:
activity = []
for k in range(1992, 2017):
  urbanization = gr.from_file("/content/drive/My Drive/Urbanization Project/datasets/urban/annual_urbanMap_global_"+str(k)+".tif")
  urbanization = urbanization.to_pandas()
  urbanization = urbanization.drop(urbanization[urbanization['value'] == 0.0].index)
  urbanization.sort_values(['x','y'], ascending=[True, True], inplace = True)
  urbanization = urbanization[['x','y']].apply(lambda x: (x / 0.25).round()* 0.25)
  urbanization = urbanization.drop(urbanization[urbanization['x'] % 0.5 == 0.0].index)
  urbanization = urbanization.drop(urbanization[urbanization['y'] % 0.5 == 0.0].index)
  urbanization.drop_duplicates(subset=['x', 'y'], inplace=True)
  urbanization.sort_values(['x','y'], ascending=[True, True], inplace = True)

  finals = ""
  finals2 = []
  iter = 0
  for i in range(-17975, 17976, 50):
    finals = ""
    for j in range(-8975, 8976, 50):
      if (urbanization['x'].loc[urbanization.index[iter]] == i/100 and urbanization['y'].loc[urbanization.index[iter]] == j/100):
        finals += "1"
        if (iter < urbanization.shape[0] - 1):
          iter+=1
      else:
        finals += "0"
    finals2.append(list(finals))
  finals2 = np.array(finals2)
  finals2 = np.rot90(finals2, k = 3, axes = (0, 1))
  finals2 = np.fliplr(finals2)

  plt.imshow(finals2.astype(float), origin='lower', extent=[6.5, 38.5, 66.5, 100], aspect='auto', cmap='coolwarm', vmin = finals2.astype(float).min(), vmax = finals2.astype(float).max()) # Convert finals2 to a float array for numerical operations
  plt.title("Urbanization in year "+str(k))
  plt.xlabel('Latitude')
  plt.ylabel('Longitude')
  plt.colorbar(label='Urbanization')
  activity.append(plt.gcf())
  plt.close()

In [ ]:
arr = [[1, 2], [3, 4]]
arr = np.array(arr)
arr_split = np.hsplit(arr, 2)[1]
arr_split2 = np.vsplit(arr_split, 2)[0]
print(arr)
print(arr_split)
arr_split2

In [ ]:
precipitation[0]

In [ ]:
forests = []
for k in range(8, 33):
  forest = gr.from_file("/content/drive/My Drive/Urbanization Project/datasets/forests/forestGain_"+str(k)+".tif")
  forest = forest.to_pandas()
  forest = forest.drop(forest[forest['value'] == 0.0].index)
  forest.sort_values(['x','y'], ascending=[True, True], inplace = True)
  forest = forest[['x','y']].apply(lambda x: (x / 0.25).round()* 0.25)
  forest = forest.drop(forest[forest['x'] % 0.5 == 0.0].index)
  forest = forest.drop(forest[forest['y'] % 0.5 == 0.0].index)
  forest.drop_duplicates(subset=['x', 'y'], inplace=True)
  forest.sort_values(['x','y'], ascending=[True, True], inplace = True)

  finals = ""
  finals2 = []
  iter = 0
  for i in range(-17975, 17976, 50):
    finals = ""
    for j in range(-8975, 8976, 50):
      if (forest['x'].loc[forest.index[iter]] == i/100 and forest['y'].loc[forest.index[iter]] == j/100):
        finals += "1"
        if (iter < forest.shape[0] - 1):
          iter+=1
      else:
        finals += "0"
    finals2.append(list(finals))
  finals2 = np.array(finals2)
  finals2 = np.rot90(finals2, k = 3, axes = (0, 1))
  finals2 = np.fliplr(finals2)

  plt.imshow(finals2.astype(float), origin='lower', extent=[min(forest['y']), max(forest['y']), min(forest['x']), max(forest['x'])], aspect='auto', cmap='coolwarm', vmin = finals2.astype(float).min(), vmax = finals2.astype(float).max()) # Convert finals2 to a float array for numerical operations
  plt.title("Urbanization in year "+str(k))
  plt.xlabel('Latitude')
  plt.ylabel('Longitude')
  plt.colorbar(label='Forest Gain')
  forests.append(plt.gcf())
  plt.close()

In [ ]:
agriculture = np.array(agriculture)
precipitation = np.array(precipitation)
temperature = np.array(temperature)
activity = np.array(activity)
forests = np.array(forests)

save_path = '/content/drive/My Drive/agriculture_array.npy'
np.save(save_path, agriculture)

save_path = '/content/drive/My Drive/urbanization_array.npy'
np.save(save_path, activity)

save_path = '/content/drive/My Drive/precipitation_array.npy'
np.save(save_path, precipitation)

save_path = '/content/drive/My Drive/temperature_array.npy'
np.save(save_path, temperature)

save_path = '/content/drive/My Drive/forests_array.npy'
np.save(save_path, forests)